# Forward and backward

In this section, we will unveil the basic mechanism of the computational process of Analytics-Zoo using a simple linear regression example. In this example, we show how to obtain the gradients with a single forward and backward pass for updating.

We first need to import the necessary modules.

In [2]:
from zoo.common.nncontext import *
from bigdl.nn.criterion import *
from zoo.pipeline.api.keras.layers import *
from zoo.pipeline.api.keras.models import *
from zoo.pipeline.api.autograd import *

import numpy as np

Using /home/lizhichao/bin/spark-2.1.0-bin-hadoop2.7
Adding /home/lizhichao/bin/god/zoo/dist/lib/zoo-0.1.0-SNAPSHOT-jar-with-dependencies.jar to BIGDL_JARS
Adding /home/lizhichao/bin/god/zoo/dist/lib/zoo-0.1.0-SNAPSHOT-jar-with-dependencies.jar to SPARK_CLASSPATH


Then we create a simple linear regression which can be formulized as *y = Wx + b*， where *W = [w1,w2]* are weight parameters and *b* is the bias.

In [13]:
# the input data size is 2*1, the output size is 1*1
linear = Sequential()
dense = Dense(1, input_shape=[2])
linear.add(dense)
# print the randomly initialized parameters
print(linear.parameters())
print(linear.get_input_shape())
print(linear.get_output_shape())

creating: createZooKerasSequential
creating: createZooKerasDense
{u'Lineard35147eb': {u'gradWeight': array([[ 0.,  0.]], dtype=float32), u'bias': array([ 0.], dtype=float32), u'weight': array([[ 0.59797329, -1.3809334 ]], dtype=float32), u'gradBias': array([ 0.], dtype=float32)}}
(None, 2)
(None, 1)


In [11]:
input = np.array([1,-2])
# forward to output
output = linear.forward(input)
print output

[ 1.62202466]


After that, we compute the error by using `AbsCrtierion` to measure the mean absolute value of the element-wise difference between input and target, then backpropagate the error of the predicted output to the input. If you want to try other criterions to compute the loss, click [here](https://bigdl-project.github.io/master/#APIGuide/Losses/) to see more details about our **Losses** API.

In [14]:
# mean absolute error
mae = AbsCriterion()
target = np.array([0])

loss = mae.forward(output, target)
print("loss: " + str(loss))
        
grad_output = mae.backward(output, target)
linear.backward(input, grad_output)

print linear.parameters()

creating: createAbsCriterion
loss: 1.6220247
{u'Lineard35147eb': {u'gradWeight': array([[ 1., -2.]], dtype=float32), u'bias': array([ 0.], dtype=float32), u'weight': array([[ 0.59797329, -1.3809334 ]], dtype=float32), u'gradBias': array([ 1.], dtype=float32)}}


From above we can see that the backward pass has computed the gradient of the weights with respect to the loss. Therefore we can update the weights with the gradients using algorithms such as *stochastic gradient descent*. However in practice you **should** use *optimizer.optimize()* to circumvent the details.